In [1]:
import os
import json

from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.series import get_series_object

import csv
from library_dicom.dicom_processor.model.Series import Series

In [2]:
series_paths = get_series_path('/home/deeplearning/AHL/AHL_Validated_DICOM')
export_folder = '/home/deeplearning/AHL/AHL_JSON'

In [3]:
#Number of serie 
len(series_paths)

504

In [4]:
index_problem = []
for serie_path in series_paths:
    print(series_paths.index(serie_path))
    try:
        dicom_serie = get_series_object(serie_path)
        dicomsInfo = dicom_serie.get_series_details()
        write_json_file(export_folder, dicomsInfo['series']['SeriesInstanceUID'], dicomsInfo)
    except Exception as err:
        print(err)
        print(dicomsInfo)
        index_problem.append(series_paths.index(serie_path))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [252]:
json_path = '/home/deeplearning/AHL/AHL_JSON'
list_dir_json = os.listdir(json_path)
list_json = []
for file_ in list_dir_json : 
    list_json.append([os.path.join(json_path, file_)])

In [253]:
for liste in list_json : 
    print(list_json.index(liste))
    with open(liste[0]) as json_file : 
        reader = json.load(json_file)

        patient_id = reader["patient"]["PatientID"]
        study_uid = reader["study"]['StudyInstanceUID']
        modal = reader['series']['Modality']

        path = reader['path']

        type_ = reader['study']["AccessionNumber"]

        liste.append(path)
        liste.append(modal)
        liste.append(type_)
        liste.append(patient_id)
        liste.append(study_uid)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [254]:
list_json[0]

['/home/deeplearning/AHL/AHL_JSON/1.2.840.113619.2.290.3.279748722.295.1383548122.747.json',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.747',
 'PT',
 'PET0',
 '13011101051024',
 '1.2.840.113619.2.290.3.279748722.295.1383548122.441']

In [255]:
#get every study_uid 
stud = []
for liste in list_json : 
    stud.append(liste[-1])

study_uid = []
study_uid.append(stud[0])
for uid in stud : 
    if uid not in study_uid : 
        study_uid.append(uid)

In [259]:
print('Nombre de study :', len(study_uid))

Nombre de study : 247


In [260]:
data = []
for uid in study_uid : 
    sub = []
    for liste in list_json : 
        if uid in liste : 
            sub.append(liste)
    data.append(sub)


In [263]:
dataset = []
for serie in data : 
    serie[0].append(serie[1][0])
    serie[0].append(serie[1][1])
    serie[0].append(serie[1][2])

    dataset.append(serie[0])



In [265]:
dataset[0]

['/home/deeplearning/AHL/AHL_JSON/1.2.840.113619.2.290.3.279748722.295.1383548122.747.json',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.747',
 'PT',
 'PET0',
 '13011101051024',
 '1.2.840.113619.2.290.3.279748722.295.1383548122.441',
 '/home/deeplearning/AHL/AHL_JSON/1.2.840.113619.2.290.3.279748722.295.1383548122.445.3.json',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.445.3',
 'CT']

In [266]:
#Clean dataset : if no json, if double study uid, if not CT&PT 

#no json 

for data in dataset : 
    if len(data) != 9 : 
        print(data)

In [267]:
#double study_uid 
double = []
double.append(dataset[0])
for data in dataset : 
    if data not in double : 
        double.append(data)

if len(double) == len(dataset) : 
    print('No double study uid')

No double study uid


In [276]:
#check Modality 
index = []
for data in dataset : 
    modal = []
    modal.append(data[2])
    modal.append(data[-1])
    print(modal)
    if modal[0] == modal[1] : 
        index.append(dataset.index(data))
print(index)
#if index is not empty : 
if len(index) != 0 :
    liste_to_remove = []
    for ind in index : 
        liste_to_remove.append(dataset[ind])

    for r in liste_to_remove : 
        dataset.remove(r)

['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['CT', 'PT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['CT', 'PT']
['PT', 'CT']
['CT', 'PT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['PT', 'CT']
['CT', 'PT']

In [26]:
#Now, can prepare json file to generate nifti in "nifti_builder_from_json.ipynb"

In [277]:
dataset[0]

['/home/deeplearning/AHL/AHL_JSON/1.2.840.113619.2.290.3.279748722.295.1383548122.747.json',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.747',
 'PT',
 'PET0',
 '13011101051024',
 '1.2.840.113619.2.290.3.279748722.295.1383548122.441',
 '/home/deeplearning/AHL/AHL_JSON/1.2.840.113619.2.290.3.279748722.295.1383548122.445.3.json',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.445.3',
 'CT']

In [279]:
study_results = []
for liste in dataset : 
    subliste = []
    subliste.append(liste[1]) #path_1
    subliste.append(liste[2]) #modal 1
    subliste.append(liste[-2]) #path_2
    subliste.append(liste[-1]) #modal_2
    subliste.append(liste[5]) #study_uid
    subliste.append(liste[3]) #type
    subliste.append(liste[4]) #patient id

    study_results.append(subliste)

In [280]:
study_results[0]

['/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.747',
 'PT',
 '/home/deeplearning/AHL/AHL_Validated_DICOM/13011101051024/pet0/1.2.840.113619.2.290.3.279748722.295.1383548122.445.3',
 'CT',
 '1.2.840.113619.2.290.3.279748722.295.1383548122.441',
 'PET0',
 '13011101051024']

In [281]:
#save results as json 
directory = '/home/deeplearning/AHL'
filename = 'AHL_new_dataset'
write_json_file(directory, filename, study_results)